<a href="https://colab.research.google.com/github/frank-lacriola/Data-Science-Lab/blob/main/Lab7%20-%20Features%20selection%20and%20classification%20for%20audio%20data/Lab7_ML_on_audios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://github.com/dbdmg/data-science-lab/raw/master/datasets/free-spoken-digit.zip

--2021-12-12 21:01:17--  https://github.com/dbdmg/data-science-lab/raw/master/datasets/free-spoken-digit.zip
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dbdmg/data-science-lab/master/datasets/free-spoken-digit.zip [following]
--2021-12-12 21:01:17--  https://raw.githubusercontent.com/dbdmg/data-science-lab/master/datasets/free-spoken-digit.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9315920 (8.9M) [application/zip]
Saving to: ‘free-spoken-digit.zip’

free-spoken-digit.z 100%[===================>]   8.88M  --.-KB/s    in 0.1s    

2021-12-12 21:01:18 (63.3 MB/s) - ‘free-spoken-dig

In [ ]:
!unzip "/content/free-spoken-digit.zip"

In [ ]:
pip install python_speech_features

  Created wheel for python-speech-features: filename=python_speech_features-0.6-py3-none-any.whl size=5888 sha256=bd9367657ed8b8af92a224d2b0b0bb10781634a360419e4bc43372c3d3e62b53
  Stored in directory: /root/.cache/pip/wheels/b0/0e/94/28cd6afa3cd5998a63eef99fe31777acd7d758f59cf24839eb
Successfully built python-speech-features


In [ ]:
import os
from scipy.io import wavfile
import numpy as np
from python_speech_features import mfcc


files_list = os.listdir("/content/dev")
labels = [f.split(".")[0].split("_")[1] for f in files_list]

In [ ]:
def pad(rate, audio):
    diff = rate - np.shape(audio)[0]
    if diff > 0:
      zero_padding = np.zeros(diff, dtype=np.float32)
      audio = np.concatenate((audio, zero_padding), axis=0)
      audio = np.reshape(audio, rate)
    else:
      audio = audio[:rate]
      
    return audio

In [ ]:
init_path = "/content/dev/"

for i, f in enumerate(files_list):
  rate, data =  wavfile.read(init_path + f)
  audio = pad(rate, data)
  feature_vector = mfcc(audio, rate)
  feature_vector = np.expand_dims(feature_vector, 0)
  if i==0:
    features = np.array(feature_vector, dtype=np.float32)
  else:
    features = np.append(features, feature_vector, axis=0)


In [ ]:
# Sklearn needs 2d inputs --> reshape
features = np.reshape(features, (features.shape[0], features.shape[1]*features.shape[2]))
print(features.shape)

(1500, 1287)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)

Let's try the trees approaches:

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print("The f1 on the test is: ", f1_score(y_pred=y_pred, y_true=y_test, average='macro'))
print("The accuracy on the test is: ", accuracy_score(y_pred=y_pred, y_true=y_test))

The f1 on the test is:  0.7253961444178388
The accuracy on the test is:  0.7333333333333333


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
print("The f1 on the test is: ", f1_score(y_pred=y_pred, y_true=y_test, average='macro'))
print("The accuracy on the test is: ", accuracy_score(y_pred=y_pred, y_true=y_test))

The f1 on the test is:  0.9698280899128356
The accuracy on the test is:  0.97


SVM multi class classification with the 'one vs one' approach:

In [ ]:
from sklearn.svm import SVC

svm = SVC(decision_function_shape='ovo') # redundant attrib.
svm.fit(X_train, y_train)

y_pred = svm.predict(X_test)

print("The f1 on the test is: ", f1_score(y_pred=y_pred, y_true=y_test, average='macro'))
print("The accuracy on the test is: ", accuracy_score(y_pred=y_pred, y_true=y_test))

The f1 on the test is:  0.9702607452244241
The accuracy on the test is:  0.97


SVM multi class classification with the 'one vs rest' approach:

In [ ]:
from sklearn.svm import LinearSVC

svm_linear = LinearSVC()
svm_linear.fit(X_train, y_train)

y_pred = svm_linear.predict(X_test)

print("The f1 on the test is: ", f1_score(y_pred=y_pred, y_true=y_test, average='macro'))
print("The accuracy on the test is: ", accuracy_score(y_pred=y_pred, y_true=y_test))

The f1 on the test is:  0.9649593483341702
The accuracy on the test is:  0.9666666666666667


Let's try cross validation over the best performing model

In [ ]:
from sklearn.model_selection import cross_val_score
import random

"""dataset = np.array(list(zip(features, labels)))
random.shuffle(dataset)"""
acc = cross_val_score(svm, features, labels, cv=5, scoring='f1_macro')
print(acc)

[0.99338436 0.98327118 0.9833852  0.98024622 0.97332323]


In [ ]:
from sklearn.model_selection import cross_val_predict

"""dataset = np.array(list(zip(features, labels)))
random.shuffle(dataset)"""
y_pred = cross_val_predict(svm, features, labels, cv=None) # default is 5-Fold
print(f1_score(y_pred, labels, average='macro'))

0.9827304685282628


Validate RF

In [ ]:
from sklearn.model_selection import cross_val_predict

"""dataset = np.array(list(zip(features, labels)))
random.shuffle(dataset)"""
y_pred = cross_val_predict(rf, features, labels, cv=None) # default is 5-Fold
print(f1_score(y_pred, labels, average='macro'))

0.970708913600682
